In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import neuroglancer
import cv2
import json

In [2]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.contour_utilities import get_contours_from_annotations, add_structure_to_neuroglancer, \
image_contour_generator
neuroglancer.set_server_bind_address(bind_port='33645')

Connecting dklab@192.168.1.12:3306


In [3]:
color_filepath = os.path.join(PATH, 'neuroglancer/contours/json_cache', 'struct_to_color.json')
with open(color_filepath, 'r') as json_file:
    structure_to_color = json.load( json_file )
    
color_segments=[]
for i in range(1,50):
    color_segments.append(i)

In [ ]:
stack = 'MD585'
detector_id = 19
# detector_id = 799

In [4]:
stack = 'MD589'
detector_id = 19
# detector_id = 799

In [ ]:
stack = 'MD594'
detector_id = 19
# detector_id = 799

# Neuroglancer code

In [5]:
viewer = neuroglancer.Viewer()
# Sets 'Image' layer to be MD585 prep2 images from S3
with viewer.txn() as s:
    s.layers[stack] = neuroglancer.ImageLayer(source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/'+stack+'_fullres')
    s.layout = 'xy' # '3d'/'4panel'/'xy'
print(viewer)

http://127.0.0.1:33645/v/d5f4e50f3383a5571f048471c6062d331bec38b3/


In [ ]:
neuroglancer.stop()

# Using Aligned Volumes

In [18]:
structure = 'SC'
# color_codes{'blue:'1,'red',2,'yellow':3}
str_contour, first_sec, last_sec = image_contour_generator( stack, detector_id, structure, use_local_alignment=True, image_prep=2, threshold=0.2)

ng_structure_volume_normal = add_structure_to_neuroglancer( viewer, str_contour, structure, stack, first_sec, last_sec, \
                                                    color_radius=5, xy_ng_resolution_um=10, threshold=0.2, color=5, \
                                                    solid_volume=False, no_offset_big_volume=False, save_results=False, \
                                                    return_with_offsets=False, add_to_ng=True, human_annotation=False)

#plt.imshow(ng_structure_volume_normal[20,:,:])
ng_structure_volume_normal.shape

atlas_structures_wrt_wholebrainWithMargin_sections [87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 2

In [ ]:
for structure in all_structures_total:
    str_contour, first_sec, last_sec = image_contour_generator( stack, detector_id, structure, use_local_alignment=True, image_prep=2, threshold=0.2)

    add_structure_to_neuroglancer( viewer, str_contour, structure, stack, first_sec, last_sec, \
                                  color_radius=5, xy_ng_resolution_um=10, threshold=0.2, color=1, \
                                  solid_volume=False, no_offset_big_volume=False, save_results=False,\
                                  return_with_offsets=False, add_to_ng=True, human_annotation=False)

In [ ]:
# CRAETE ENTIRE BRAIN VOLUME
xy_ng_resolution_um = 5

with open('struct_reverse_2.json', 'r') as json_file:
    structure_to_color = json.load( json_file )

    
# MD585: x_um = 35617,           y_um = 26086
# MD585: x_pixels_.46res = x_um*0.46,  y_pixels_.46res = y_um*0.46
# MD585: x_pixels_newres = x_pixels_.46res*(0.46/newres), y_pixels_newres = y_pixels_.46res*(0.46/newres)
# microns/resolution
y_voxels = int( 26086*0.46*(.46/xy_ng_resolution_um) + 0.5)
x_voxels = int( 35617*0.46*(.46/xy_ng_resolution_um) + 0.5)
full_brain_volumes = np.zeros((268,y_voxels,x_voxels), dtype=np.uint8)

for structure in all_structures_total:
    str_contour, first_sec, last_sec = image_contour_generator( stack, detector_id, structure, use_local_alignment=True, image_prep=2, threshold=0.5)
    
    try:
        color=structure_to_color[structure]
    except:
        color=2
    
    str_volume, xyz_offsets = add_structure_to_neuroglancer( viewer, str_contour, structure, stack, first_sec, last_sec, \
                                          color_radius=5, xy_ng_resolution_um=xy_ng_resolution_um, threshold=0.5, color=color, \
                                          solid_volume=False, no_offset_big_volume=True, save_results=False, return_with_offsets=True, \
                                          add_to_ng=False, human_annotation=False )
    
    z_len, y_len, x_len = np.shape(str_volume)
    full_brain_volumes[0:z_len, 0:y_len, 0:x_len] += str_volume



color_segments=[]
for i in range(1,50):
    color_segments.append(i)

with viewer.txn() as s:
    #s.layers[ display_name ] = neuroglancer.SegmentationLayer(
    s.layers[ "full_brain" ] = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(
            data=full_brain_volumes, # Z,Y,X
            voxel_size=[ xy_ng_resolution_um*1000, xy_ng_resolution_um*1000,20000], # X Y Z
            voxel_offset = [ 0, 0, 0] # X Y Z
        ),
        segments = color_segments
    )

In [ ]:
color_array = np.ones((1,80,80), dtype=np.uint8)
color_array[0,0:49,0]= np.array(color_segments)
color_array[0,0,0:49]= np.array(color_segments)


with viewer.txn() as s:
    #s.layers[ display_name ] = neuroglancer.SegmentationLayer(
    s.layers[ "color_test" ] = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(
            data= color_array, # Z,Y,X
            voxel_size=[ 50000, 50000,200000], # X Y Z
            voxel_offset = [ 0, 0, 0] # X Y Z
        ),
        segments = color_segments
    )

# Human Annotation Contours

In [ ]:
! ls $ROOT_DIR/CSHL_labelings_v3/MD585/MD585_annotation_contours_05302018161849.hdf

In [ ]:
! ls $ROOT_DIR/CSHL_labelings_v3/$stack/

In [ ]:
! ls $ROOT_DIR/CSHL_labelings_v3/MD585/

In [ ]:
! ls $ROOT_DIR/CSHL_labelings_v3/MD589/

In [ ]:
! ls $ROOT_DIR/CSHL_labelings_v3/MD594/

In [ ]:
hand_annotations

In [ ]:

# MD585
if stack=="MD585":
    hand_annotations = load_hdf_v2(ROOT_DIR+'/CSHL_labelings_v3/MD585/MD585_annotation_contours_05302018161849.hdf')
if stack=="MD589":
    hand_annotations = load_hdf_v2(ROOT_DIR+'/CSHL_labelings_v3/MD589/MD589_annotation_contours_05302018164231.hdf')
if stack=="MD594":
    hand_annotations = load_hdf_v2(ROOT_DIR+'/CSHL_labelings_v3/MD594/MD594_annotation_contours_05312018035134.hdf')
    
# Converts thw weird coordinate frame to a normal-ass prep2 raw frame
hand_annotations = convert_annotation_v3_original_to_aligned_cropped_v2(\
                                        hand_annotations, stack=stack,\
                                        out_resolution='raw',
                                        prep_id=2)


num_annotations = len(hand_annotations)
#hand_annotations.keys()


MD585_ng_section_min = 83
MD585_ng_section_max = 536
# For sections before the 22nd section there is a weird offset that this compensates for
MD585_abberation_correction = [9872, 4258] # UNUSED Currently

def get_dense_coordinates( coor_list ):
    dense_coor_list = []
    # Shortest distance, x, y

    #for x, y in coor_list:
    for i in range(len(coor_list)-1):
        x, y = coor_list[i]
        x_next, y_next = coor_list[i+1]
        
        x_mid = (x+x_next)/2
        y_mid = (y+y_next)/2
        
        
        dense_coor_list.append([x,y])
        dense_coor_list.append( [x_mid, y_mid] )
        
        if i==len(coor_list)-2:
            dense_coor_list.append( [x_next, y_next] )
            x, y = coor_list[0]
            x_mid = (x+x_next)/2
            y_mid = (y+y_next)/2
            dense_coor_list.append( [x_mid, y_mid] )
        
    return dense_coor_list

def get_contours_from_annotations( stack, target_str, densify=0 ):
    str_contours_annotation = {}

    for i in range(num_annotations):
        structure = hand_annotations['name'][i]
        side = hand_annotations['side'][i]
        section = hand_annotations['section'][i]
        
        if side=='R' or side=='L':
            structure = structure+'_'+side
            
        if structure==target_str:
            vertices = hand_annotations['vertices'][i]
            
            for i in range(densify):
                vertices = get_dense_coordinates( vertices )
            
            # Skip sections before the 22nd prep2 section for MD585 as there are clear errors
            if stack=='MD585' and section < MD585_ng_section_min+22:
                #vertices = vertices - np.array(MD585_abberation_correction)
                continue

            str_contours_annotation[section] = {}
            str_contours_annotation[section][structure] = {}
            str_contours_annotation[section][structure][1] = vertices

    first_sec = np.min(str_contours_annotation.keys())
    last_sec = np.max(str_contours_annotation.keys())
    
    return str_contours_annotation, first_sec, last_sec

In [ ]:
# for target_str in all_structures_total[0:3]:
for target_str in ['12N']:
    str_contours_annotation, first_sec, last_sec = get_contours_from_annotations( stack, target_str, densify=4 )

    ng_structure_volume = add_structure_to_neuroglancer( viewer, str_contours_annotation, target_str, stack, first_sec, last_sec, \
                                                    color_radius=3, xy_ng_resolution_um=10, threshold=1, color=4, \
                                                    solid_volume=False, no_offset_big_volume=False, save_results=False, \
                                                    return_with_offsets=False, add_to_ng=True, human_annotation=True )

In [ ]:
# for target_str in all_structures_total[0:3]:
for target_str in ['7n_R']:
    str_contours_annotation, first_sec, last_sec = get_contours_from_annotations( stack, target_str, densify=4 )

    ng_structure_volume = add_structure_to_neuroglancer( viewer, str_contours_annotation, target_str, stack, first_sec, last_sec, \
                                                    color_radius=3, xy_ng_resolution_um=10, threshold=1, color=4, \
                                                    solid_volume=False, no_offset_big_volume=False, save_results=False, \
                                                    return_with_offsets=False, add_to_ng=True, human_annotation=True )

In [ ]:
# for target_str in all_structures_total[0:3]:
for target_str in ['7n_R']:
    str_contours_annotation, first_sec, last_sec = get_contours_from_annotations( stack, target_str, densify=4 )

    ng_structure_volume = add_structure_to_neuroglancer( viewer, str_contours_annotation, target_str, stack, first_sec, last_sec, \
                                                    color_radius=3, xy_ng_resolution_um=10, threshold=1, color=4, \
                                                    solid_volume=False, no_offset_big_volume=True, save_results=False, \
                                                    return_with_offsets=False, add_to_ng=True, human_annotation=True )

In [ ]:
# REAL
# 16384 x 12000 pixels raw
# 35617 x 26086 um raw

In [ ]:
x_voxels

In [ ]:
# Add all annotated brains to the viewer
xy_ng_resolution_um = 5
color_radius = 3

# MD585: x_um = 35617,           y_um = 26086
# MD585: x_pixels_.46res = x_um*0.46,  y_pixels_.46res = y_um*0.46
# MD585: x_pixels_newres = x_pixels_.46res*(0.46/newres), y_pixels_newres = y_pixels_.46res*(0.46/newres)
# microns/resolution
y_voxels = 1+int( 26086*0.46*(.46/xy_ng_resolution_um) + 0.5)
x_voxels = 1+int( 35617*0.46*(.46/xy_ng_resolution_um) + 0.5)
full_brain_volume_annotated = np.zeros((268,y_voxels,x_voxels), dtype=np.uint8)

for target_str in all_structures_total:
# for target_str in['VCA_L','7n_R','7n_L']:
    print(target_str)
    str_contours_annotation, first_sec, last_sec = get_contours_from_annotations( stack, target_str, densify=4 )
    
    try:
        color=structure_to_color[target_str]
    except Exception as e:
        print(e)
        color=4
    
    str_volume, xyz_str_offsets = add_structure_to_neuroglancer( \
                            viewer, str_contours_annotation, target_str, stack, first_sec, last_sec, \
                            color_radius=color_radius, xy_ng_resolution_um=xy_ng_resolution_um, threshold=1, color=color, \
                            solid_volume=False, no_offset_big_volume=True, save_results=True, \
                            return_with_offsets=True, add_to_ng=False, human_annotation=True  )
    
    z_len, y_len, x_len = np.shape(str_volume)
#     full_brain_volume_annotated[0:z_len, 0:y_len, 0:x_len] = str_volume.copy()
    for z in range( xyz_str_offsets[2], z_len ):
        for y in range( xyz_str_offsets[1], y_len ):
            for x in range( xyz_str_offsets[0], x_len ):
                structure_val = str_volume[z, y, x]
                if structure_val==0:
                    continue
                else:
                    try:
                        full_brain_volume_annotated[z, y, x] = structure_val
                    except Exception as e:
                        print(e)
    
with viewer.txn() as s:
    s.layers[ stack+"_Atlas" ] = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(
            data=full_brain_volume_annotated, # Z,Y,X
            voxel_size=[ xy_ng_resolution_um*1000, xy_ng_resolution_um*1000,20000], # X Y Z
            voxel_offset = [ 0, 0, 0] # X Y Z
        ),
        segments = color_segments
    )

In [ ]:
with viewer.txn() as s:
    s.layers[ stack+"_Atlas" ] = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(
            data=full_brain_volume_annotated, # Z,Y,X
            voxel_size=[ xy_ng_resolution_um*1000, xy_ng_resolution_um*1000,20000], # X Y Z
            voxel_offset = [ 0, 0, 0] # X Y Z
        ),
        segments = color_segments
    )

In [ ]:
full_annotation_fp = NEUROGLANCER_ROOT+stack+'/human_annotation/solid_volume_'+str(xy_ng_resolution_um)+'um/'
full_annotation_fn = full_annotation_fp+'volume_colored.npy'
print('Saving to :'+full_annotation_fp+full_annotation_fn)

if not os.path.exists( full_annotation_fp ):
    os.makedirs(full_annotation_fp)
    
np.save( full_annotation_fn, full_brain_volume_annotated )
# radius <= 1 : wire
#>radius <= 2 : thin
#>radius <= 3.5 : ''

In [ ]:
! mkdir /home/alexn/Desktop/neuroglancer_binary_volumes/human_annotations_5um/

# TESTING

In [ ]:
! aws s3 rm --recursive s3://test-bucket-sid/alex_neuroglancer_volumes

In [ ]:
! aws s3 cp --recursive $NEUROGLANCER_ROOT s3://test-bucket-sid/alex_neuroglancer_volumes

In [ ]:
os.environ['ROOT_DIR']

# End Neuroglancer Code

In [ ]:
# Resets the viewer
with viewer.txn() as s:
    # Resets X/Y/Z plane orientation
    s.navigation.pose.orientation = [0,0,0,1]
    # Zooms out 
    s.navigation.zoomFactor = 10000
    
    # Resets 3D Viewer Orientation
    s.perspectiveOrientation = [0,0,0,1]
    # Zooms out
    s.perspectiveZoom = 75000
    
    # Not necessary, just restates the voxel sizes of the image
    s.navigation.pose.position.voxelSize = [460,460,20000]
    # Sets Viewer's center location
    s.navigation.pose.position.voxelCoordinates = [8192,6000,134]

In [ ]:
# wholeslice_to_brainstem = -from_padded_to_wholeslice, from_padded_to_brainstem

#from_padded_to_wholeslice
rostral_limit = 50
caudal_limit = 1188
dorsal_limit = 21
ventral_limit = 738

#from_padded_to_brainstem
rostral_limit = 521
caudal_limit = 1057
dorsal_limit = 128
ventral_limit = 465

In [ ]:
fp_volumes = '/home/alexn/Desktop/neuroglancer_binary_volumes/volumes_'+xy_ng_resolution_um+'um/'
np.save( fp_volumes+structure+'_volume.npy',structure_volume)